# Data Preparation
<hr>
This notebook contains the code used to prepare the datasets that were required for the various business questions.

In [2]:
# Library imports
import pandas as pd; import numpy as np; import matplotlib.pyplot as plt

%matplotlib inline

## 1. How does the price change in different areas based on seasonality?

In [3]:
# Read in datasets
listings_full_df = pd.read_csv("../Data/listings.csv.gz", low_memory=False)

### Prepare listings data

In [6]:
# Which columns have high percentages of missing data
cols_null_50 = listings_full_df.columns[listings_full_df.isnull().mean() > .5].tolist()
cols_null_75 = listings_full_df.columns[listings_full_df.isnull().mean() > .75].tolist()
cols_null_100 = listings_full_df.columns[listings_full_df.isnull().mean() == 1.].tolist()

In [9]:
# Remove all vars that have 75% missing or above
listings_full_df = listings_full_df.drop(cols_null_75, axis=1)

In [10]:
# Which columns have no missing values
cols_no_nulls = listings_full_df.columns[listings_full_df.isnull().mean() == 0].tolist()

In [11]:
# Which columns have low cardinality
cols_low_cardinality = listings_full_df.columns[listings_full_df.nunique() == 1].tolist()

In [12]:
# Remove columns with low cardinality
listings_full_df = listings_full_df.drop(cols_low_cardinality, axis=1)

Identify the columns needed to answer the time of season business question

In [13]:
listings_full_df.columns

Index(['id', 'listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'neighborhood_overview', 'transit', 'access',
       'interaction', 'house_rules', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed', 'city', 'state', 'zipcode',
       'market', 'smart_location', 'country_code', 'country', 'latitude',
       'longitude', 'is_location_exact', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'amenities', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 

In [75]:
# Get columns needed from listings data
listings_1_df = listings_full_df[['id', 'neighbourhood', 'neighbourhood_cleansed', 'smart_location', 'property_type', 'room_type']].copy()

european_n = ['Fatih', 'Zeytinburnu', 'Bakirkoy', 'Kucukcekmece', 'Avcilar', 'Bahcelievler', 'Arnavutkoy', 'Bagcilar', 'Basaksehir', 'Bayrampasar', 'Besiktas',
             'Beylikduzu', 'Beyoglu', 'Buyukcekmece', 'Catalca', 'Esenler', 'Esenyurt', 'Eyup', 'Gaziosmanpasa', 'Gungoren', 'Kagithane', 'Sariyer', 'Silivri', 'Sisli',
             'Sultangazi']
asian_n = ['Atasehir', 'Beykoz', 'Cekmekoy', 'Kadikoy', 'Kartal', 'Maltepe', 'Pendik', 'Sancaktepe', 'Sile', 'Sultanbeyli', 'Tuzla', 'Umraniye', 'Uskudar']
listings_1_df['Side'] = listings_1_df.neighbourhood_cleansed.apply(lambda x: 'European' if x in european_n else 'Asian')

### Prepare calendar data

In [14]:
# Read in dataset
calendar_df = pd.read_csv("../Data/calendar.csv.gz", low_memory=False)

In [42]:
# Filter out listings that have a min nights stay greater than 5
calendar_fil_df = calendar_df.query("minimum_nights <= 5").copy()

In [44]:
# Convert date feature to date type and create new columns
calendar_fil_df.date = pd.to_datetime(calendar_fil_df.date)
calendar_fil_df['Month'] = calendar_fil_df.date.dt.month
calendar_fil_df['MonthString'] = calendar_fil_df.Month.map({1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct',
                                                           11: 'Nov', 12: 'Dec'})
calendar_fil_df['Year'] = calendar_fil_df.date.dt.year

In [52]:
# Filter ids that have missing price data and investigate
id_w_missing_prices = calendar_fil_df.loc[calendar_fil_df.isnull().any(axis=1), 'listing_id'].unique().tolist()

In [54]:
# Remove rows with missing price data
calendar_fil_df = calendar_fil_df.dropna(subset=['price', 'adjusted_price'], how='any', axis=0)

In [56]:
# Create new column that converts price to float type
calendar_fil_df['adjusted_price_num'] = calendar_fil_df.adjusted_price.str.replace('[$,]', '').astype(float)

In [59]:
# Create pivot table showing price for each month for each listing
calendar_price_df = calendar_fil_df.pivot_table(index='listing_id', columns='MonthString', values='adjusted_price_num', aggfunc='mean').reset_index()
calendar_price_df = calendar_price_df[['listing_id', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

In [63]:
calendar_price_df.head()

MonthString,listing_id,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,4826,734.0,734.0,734.0,734.0,734.000000,734.0,734.0,734.0,734.0,734.0,734.0,734.0
1,28277,749.0,749.0,749.0,749.0,749.000000,749.0,749.0,749.0,749.0,749.0,749.0,749.0
2,28318,872.0,872.0,872.0,872.0,872.000000,872.0,872.0,872.0,872.0,872.0,872.0,872.0
3,29241,349.0,349.0,349.0,349.0,349.000000,349.0,349.0,349.0,349.0,349.0,349.0,349.0
4,29692,6512.0,6512.0,6512.0,6512.0,6304.387097,5921.4,6512.0,6512.0,6512.0,6512.0,6512.0,6512.0


### Merge prepared listings data with prepared calendar data

In [76]:
final_df_1 = listings_1_df.merge(calendar_price_df, how='inner', left_on='id', right_on='listing_id').drop(['listing_id'], axis=1)

In [77]:
final_df_1.to_csv('../Data/DataPrepQ1.csv', index=False)

## 2. Which areas to stay and avoid

In [79]:
listings_full_df.columns

Index(['id', 'listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'neighborhood_overview', 'transit', 'access',
       'interaction', 'house_rules', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed', 'city', 'state', 'zipcode',
       'market', 'smart_location', 'country_code', 'country', 'latitude',
       'longitude', 'is_location_exact', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'amenities', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 

In [84]:
listings_2_df = listings_full_df[['id', 'summary', 'description', 'neighbourhood_cleansed', 'review_scores_rating']]
listings_2_df.head()

,id,summary,description,neighbourhood_cleansed,review_scores_rating
0,4826,My place is close to great views. My place is ...,My place is close to great views. My place is ...,Uskudar,100.0
1,20815,Watch The Bosphorus from The Comfy Hill. A spa...,Watch The Bosphorus from The Comfy Hill. A spa...,Besiktas,90.0
2,27271,This is a very nicely decorated apartment in a...,This is a very nicely decorated apartment in a...,Beyoglu,98.0
3,28277,NaN,The first advantage to stay in our apartments ...,Sisli,NaN
4,28318,NaN,Easy access to both bridges and just next to F...,Sariyer,NaN


In [82]:
reviews_df = pd.read_csv("../Data/reviews.csv.gz", low_memory=False)

In [83]:
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,4826,3419,2009-06-01,13823,Anja,We just came back from a weeks stay at Kaan's ...
1,20815,31174,2010-03-24,49388,Chuck,My daughter and her friend and I all stayed at...
2,20815,40738,2010-05-08,93598,Sara,You cannot find a better place to stay in Turk...
3,20815,49182,2010-06-02,108540,Suzan,This comfortable apartment truly deserves the ...
4,20815,85462,2010-08-27,126482,William,I second all the excellent reviews of this pla...


In [86]:
final_df_2 = reviews_df.merge(listings_2_df, how='inner', left_on='listing_id', right_on='id').drop(['id_x', 'id_y'], axis=1)

In [88]:
final_df_2.to_csv("../Data/DataPrepQ2.csv", index=False)